# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-3362000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-2082000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Bidding

In [2]:
vuln_ns, vuln_ew = False, False
hand = 'AQJ986.QJ952.7.8'

auction = ["PASS", "1N"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

['PASS', '1N']
Bid not valid: 1S insta_score: 0.076 0.07


FileNotFoundError: Could not find module 'libdds.so' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


West always has the same cards because it's your hand and it's known to you

In [ ]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction, 2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')